In [1]:
import os, sys
import ipdb # debugging
from tqdm import tqdm # dat
from datetime import datetime # may use for checkpoint files for the state of training
import platform, shutil # detect platform type
import requests, zipfile, io


# Pytorch
import torch # library for deep learning
import torch.nn as nn # neuron network
from torch.nn import functional as F

# Tokenizer
import sentencepiece as spm # translate human language to tokens for computer to understand

# A-series GPU will benefit from this setting
#torch.backends.cuda.matmul.allow_tf32 = True
#torch.backends.cudnn.allow_tf32 = True

torch.cuda.empty_cache() # empty GPU cache
files_url = "https://ideami.com/llm_train"
print("Downloading files using Python")
response = requests.get(files_url) # GET request to fetch stuff from the website
zipfile.ZipFile(io.BytesIO(response.content)).extractall(".") # . indicate current folder





In [4]:

# ARCHITECTURE PARAMETERS
batch_size = 32 # 8 to 128 and beyond, if only 4GB available for your GPU then make this 8 -- GPU for parallel computing
context = 512 # Sequence length aka focus window
embed_size = 384 # dimensions of vectors, this means 384 numbers to explain a token
n_layers = 7
n_heads = 7
BIAS = True

# Hyper parameters are the ones impacting the training process
lr = 3e-4
dropout = 0.05
weight_decay = 0.01
grad_clip = 1.0

# Training parameters
load_pretrained = True
train_iters = 100000
eval_interval = 50 # Every 50 times of iteration we are going to evaluate our data
eval_iters = 10 
compile = True
checkpoint_dir = "models/"
# checkpoint_fn = "lastest.pt"
checkpoint_fn = 'latest.pt' # File name for saving a checkpoint
# checkpoint_load_fn = "latest.pt"
checkpoint_load_fn = 'latest.pt' # File name for loading a checkpoint
dtype = torch.bfloat16

# Mode : False for training, True for getting output for use 
inference = True

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device: You will be using: ", device)





device: You will be using:  cuda


In [5]:
# Logging

wandb_log = True
wandb_project = "llm9"
# wandb_run_name = "llm1-" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
wandb_run_name = 'llm9run'

if wandb_log:
    import wandb # For deep-learning metrics visualizaiton
    wandb.init(project = wandb_project, name = wandb_run_name)

#wandb.finish() # This will finish the project






wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jwzero8 (jwzero8-the-university-of-hong-kong). Use `wandb login --relogin` to force relogin


In [6]:
# Load Data
with open('wiki.txt', 'r', encoding = 'utf-8') as f: #
    text = f.read()
print(text[30000:30300])









terms.
For example, there are objects in two groups (as shown on the right). The objects are various shapes, where one group has 3 of them while the other has 2. When the two groups combine into one, the overall amount (sum) of the shapes become 5.

Vertical Addition

The animation above demonstrate


In [7]:
# Initialize Tokenizer

sp = spm.SentencePieceProcessor(model_file = 'wiki_tokenizer.model')

vocab_size = sp.get_piece_size()
print(f"Tokenizer vocab size: {vocab_size}")




Tokenizer vocab size: 4096


In [8]:
# Set up tokenizing function
encode = lambda s : sp.Encode(s)
decode = lambda l : sp.Decode(l)

print(encode('Once upon a time'))
print(decode(encode('Once upon a time')))
      

[612, 370, 698, 265, 261, 684]
Once upon a time


In [9]:
# Tokenize our data or load tokenizer if pre-trained tokenizers are available
if os.path.exists('encoded_data.pt'):
    print('Loading Encoding')
    data = torch.load('encoded_data.pt')
else:
    data = torch.tensor(encode(text), dtype = torch.long) # dtype can be more accurate for better performacne yet more memory and computation power is required
    torch.save(data, 'encoded_data.pt')




Loading Encoding


C:\Users\ZGMF-X42S\AppData\Local\Temp\ipykernel_2036\2177634386.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('encoded_data.pt')


In [10]:
# Split data between traiing and evaluation datasets
data_size = len(data)
spl = int(0.9 * data_size)
train_data = data[:spl]
val_data = data[spl:]
print(f'Total size: {data_size/1e6:.2f} million | Training {len(train_data)/1e6:.2f} Million | Validation {len(val_data)/1e6:.2f} Million')




Total size: 59.21 million | Training 53.29 Million | Validation 5.92 Million


In [11]:
def get_batch(split):
    # BS = Batch Size (e.g. 32) / SL = Sequence or Context Length (e.g. 512)
    data = train_data if split == 'train' else val_data
    inds = torch.randint(len(data) - context, (batch_size,))
    x = torch.stack([data[i: i + context] for i in inds]) # (BS, SL) e.g. (32, 512)
    y = torch.stack([data[i + 1: i + context + 1] for i in inds])

    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch("train")
print(x.shape, y.shape)
print(x[0][:10])
print(y[0][:10])
    





torch.Size([32, 512]) torch.Size([32, 512])
tensor([ 410, 4051,  310,  353,  944,  836,  381, 4035,  307, 4031],
       device='cuda:0')
tensor([4051,  310,  353,  944,  836,  381, 4035,  307, 4031, 4056],
       device='cuda:0')


In [12]:
class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size) # In our case will be from 4096 x 384
        self.positions = nn.Embedding(context, embed_size) # e.g. 512 x 384
        # Number of heads with multi-attention mechanism in each layer, which is 7 in this case
        self.blocks = nn.Sequential(*[Block(n_heads) for _ in range(n_layers)]) 
        self.ln = nn.LayerNorm(embed_size)
        self.final_linear = nn.Linear(embed_size, vocab_size, bias=BIAS) # e.g. 384 x 4906, {embeddings x vocab tokens/options}
        self.apply(self._init_weights)
    
    # parameter initialization
    def _init_weights(self, module):
        if isinstance(module, nn.Linear): # If it is linear layer
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)


    def forward(self, input, targets = None):
        # BS = batch size, SL = Sequence or Context Length
        loss = None
        BS, SL = input.shape # BS x SL
        emb = self.embeddings(input) # BS x SL x Embed_size(384)
        pos = self.positions(torch.arange(SL, device = device)) # SL x 384
        x = emb + pos # BS x SL x 384
        x = self.blocks(x) # Pass our input to blocks, traverl through blocks and layers, BS x SL x 384
        x = self.ln(x) # BS x SL x 384 (Embed Size)
        logits = self.final_linear(x) # BS x SL x 4096 (vocab size)



        if targets is not None:
            BS, SL, VS = logits.shape # BS x SL x 4906 (Vocabulary Size)
            logits = logits.view(BS*SL, VS)
            targets = targets.view(BS*SL)
            loss = F.cross_entropy(logits, targets)     


            # Manual calculation 
            counts = logits.exp()
            prob = counts / counts.sum(-1, keepdim = True) # Keep dim = keeping the dimension for dividing each single dimension
            loss2 = prob[torch.arange(BS*SL), targets].log().mean()
            # E.g., Target[3] = 329 prob[3][329] = 0.014
            # Cross entropy = - Log p(x)

            # if (not torch.allclose(loss, loss2)): # This is to show Pytorch calculate entropy is a bit different from manual way, on a time-to-time basis
            #    print(f"[Loss Diff] Pytorch:{loss.item()} Manual:{loss2.item()}")
                
        return logits, loss

        # Generate a new sample
    def generate(self, input, max = 500):
        for _ in range(max):
            input = input[:, -context:] # (1, input length until max of SL)
            logits, _ = self(input) # (1, input length, 4096)
            logits = logits[:, -1,:] # Pick last logits/probability (1, 4096) as we are extracting the last token
            probs = F.softmax(logits, dim = -1) # (1, 4096)
            next = torch.multinomial(probs, num_samples = 1)
            input = torch.cat((input, next), dim = 1)
        return input
                



        
        
            








In [13]:
class Block(nn.Module):
    def __init__(self, n_heads):
        super().__init__()
        head_size = embed_size // n_heads
        self.ma = Multihead(n_heads, head_size)
        self.feed_forward = ForwardLayer(embed_size)
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)


    def forward(self, x):
        x = x + self.ma(self.ln1(x))
        x = x + self.feed_forward(self.ln2(x))
        return x






In [14]:
class ForwardLayer(nn.Module):
    def __init__(self, embed_size):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(embed_size, 6 * embed_size, bias = BIAS),
        nn.GELU(),
        nn.Linear(6 * embed_size, embed_size, bias = BIAS),
        nn.Dropout(dropout)
    )

    def forward(self, x):
        x = self.network(x)
        return x 



In [15]:
class Multihead(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.combine = nn.Linear(head_size * n_heads, embed_size, bias = BIAS) # 378, 384
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = torch.cat([head(x) for head in self.heads], dim = -1)
        # Each head output (BS, SL, head_size)
        x = self.combine(x)
        x = self.dropout(x)
        return x

In [16]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.queries = nn.Linear(embed_size, head_size, bias = BIAS)
        self.keys = nn.Linear(embed_size, head_size, bias = BIAS)
        self.values = nn.Linear(embed_size, head_size, bias = BIAS)

        self.register_buffer('tril', torch.tril(torch.ones(context, context)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        BS, SL, VS = x.shape
        q = self.queries(x) # BS, SL, 54
        k = self.keys(x) # BS, SL, 54
        v = self.values(x) # BS, SL, 54

        attn_w = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # BS, SL, SL
        attn_w = attn_w.masked_fill(self.tril[:SL, :SL] == 0, float('-inf'))
        attn_w = F.softmax(attn_w, dim = -1) # BS, SL, SL

        x = attn_w @ v # BS, SL, 54

        return x
        

In [17]:
head_size = embed_size // n_heads
print(f"embed: {embed_size} n_heads: {n_heads} head_size: {head_size}")

embed: 384 n_heads: 7 head_size: 54


In [52]:
# Understand the attention calculations
# Commented out as it is for learning purpose 

'''
x, y = get_batch('train')
print(x.shape, y.shape)
print(x[0][:10])
print(y[0][:10])
x = x.to(device)
y = y.to(device)

embeddings = nn.Embedding(vocab_size, embed_size).to(device)
positions = nn.Embedding(context, embed_size).to(device)
queries = nn.Linear(embed_size, head_size, bias = BIAS).to(device)
keys = nn.Linear(embed_size, head_size, bias = BIAS).to(device)
values = nn.Linear(embed_size, head_size, bias = BIAS).to(device)
tril = torch.tril(torch.ones(context, context)).to(device)

emb = embeddings(x)
pos = positions(torch.arange(context, device = device))
x = emb + pos

q = queries(x)
k = keys(x)
v = values(x)
print(q.shape, k.shape, v.shape)
torch.set_printoptions(precision = 2, sci_mode = False)
print(q[0][0][:5])

attn_w = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
attn_w = attn_w.masked_fill(tril[:context, :context] == 0, float('-inf'))
attn_w = F.softmax(attn_w, dim = -1)
x = attn_w @ v
'''

"\nx, y = get_batch('train')\nprint(x.shape, y.shape)\nprint(x[0][:10])\nprint(y[0][:10])\nx = x.to(device)\ny = y.to(device)\n\nembeddings = nn.Embedding(vocab_size, embed_size).to(device)\npositions = nn.Embedding(context, embed_size).to(device)\nqueries = nn.Linear(embed_size, head_size, bias = BIAS).to(device)\nkeys = nn.Linear(embed_size, head_size, bias = BIAS).to(device)\nvalues = nn.Linear(embed_size, head_size, bias = BIAS).to(device)\ntril = torch.tril(torch.ones(context, context)).to(device)\n\nemb = embeddings(x)\npos = positions(torch.arange(context, device = device))\nx = emb + pos\n\nq = queries(x)\nk = keys(x)\nv = values(x)\nprint(q.shape, k.shape, v.shape)\ntorch.set_printoptions(precision = 2, sci_mode = False)\nprint(q[0][0][:5])\n\nattn_w = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5\nattn_w = attn_w.masked_fill(tril[:context, :context] == 0, float('-inf'))\nattn_w = F.softmax(attn_w, dim = -1)\nx = attn_w @ v\n"

In [53]:
# Understand Attention Matrix
# Commented out as it is for learning purpose 
'''
full = q @ k.transpose(-2, -1) # 512*54 @ 54 x 512 = 512 x 512

a = q[0][5]
b = k.transpose(-2, -1)[0, :, 3]
print(a, b)
c = torch.dot(a, b)
print(c)
print(full[0][5][3])
'''

'\nfull = q @ k.transpose(-2, -1) # 512*54 @ 54 x 512 = 512 x 512\n\na = q[0][5]\nb = k.transpose(-2, -1)[0, :, 3]\nprint(a, b)\nc = torch.dot(a, b)\nprint(c)\nprint(full[0][5][3])\n'

In [54]:
# Understand the updating of the V content
# Commented out as it is for learning purpose 
'''
print(attn_w.shape, v.shape)

print(x[0][7])

attn_scores2 = attn_w[0, 7, :] # Shape [512]

# Initialize a tensor to sotre the result
result = torch.zeros(54)
# Compute the dot product for each column in V for the first tokens in the first batch
for i in range(54):
    result[i] = torch.dot(attn_scores2, v[0, :, i])

print(result)
'''


'\nprint(attn_w.shape, v.shape)\n\nprint(x[0][7])\n\nattn_scores2 = attn_w[0, 7, :] # Shape [512]\n\n# Initialize a tensor to sotre the result\nresult = torch.zeros(54)\n# Compute the dot product for each column in V for the first tokens in the first batch\nfor i in range(54):\n    result[i] = torch.dot(attn_scores2, v[0, :, i])\n\nprint(result)\n'

In [18]:
# Optional (after doing it, take out the loss2 from the output of the model)
x, y = get_batch('train')
print(x.shape, y.shape)
print(x[0][:10])
print(y[0][:10])

model = GPT()
model = model.to(dtype) # Denote the precision level
model = model.to(device)


# logit, loss, loss2 = model(x, y)
# print(loss.item(), loss2.item())

torch.Size([32, 512]) torch.Size([32, 512])
tensor([4055,  307,  590,  337,  430, 1248, 4042, 2087,  437, 1675],
       device='cuda:0')
tensor([ 307,  590,  337,  430, 1248, 4042, 2087,  437, 1675,  441],
       device='cuda:0')


In [19]:
@torch.no_grad()
def generate_sample(input):
    t1 = torch.tensor(encode(input), dtype = torch.long, device = device) # take input, encode it, transform to tensor
    t1 = t1[None, :] # (1, [size of the ids]  - BS = 1 as we are inferencing
    newgen = model.generate(t1, max = 64)[0].tolist()
    result = decode(newgen)
    print(f"{result}")

# generate_sample('I am a person.')




In [20]:
# Training Setup


model = GPT()
model = model.to(dtype)
model = model.to(device)

if compile:
    print('Torch: Compiling model')
    model = torch.compile (model)

print(sum(p.numel() for p in model.parameters()) / 1e6, 'Million parameters')








Torch: Compiling model
19.837954 Million parameters


In [21]:
# Calculate loss averages
import torch._dynamo
torch._dynamo.config.suppress_errors = True

@torch.no_grad()
def calculate_loss():
# WE are evaluating but not training so this decorator
    out = {}
    model.eval()
    for split in ['train', 'eval']:
        l = torch.zeros(eval_iters)
        # calculation the MEAN of the loss
        for i in range(eval_iters):
            x, y = get_batch(split)
            _, loss = model(x, y)
            l[i] = loss
        out[split] = l.mean().item()
    model.train()
    return out

l = calculate_loss()
print(l)
            









W1029 11:03:09.496000 2036 site-packages\torch\_dynamo\convert_frame.py:1125] WON'T CONVERT forward C:\Users\ZGMF-X42S\AppData\Local\Temp\ipykernel_2036\3703986496.py line 23 
W1029 11:03:09.496000 2036 site-packages\torch\_dynamo\convert_frame.py:1125] due to: 
W1029 11:03:09.496000 2036 site-packages\torch\_dynamo\convert_frame.py:1125] Traceback (most recent call last):
W1029 11:03:09.496000 2036 site-packages\torch\_dynamo\convert_frame.py:1125]   File "C:\Users\ZGMF-X42S\miniconda3\envs\test2\Lib\site-packages\torch\_dynamo\output_graph.py", line 1446, in _call_user_compiler
W1029 11:03:09.496000 2036 site-packages\torch\_dynamo\convert_frame.py:1125]     compiled_fn = compiler_fn(gm, self.example_inputs())
W1029 11:03:09.496000 2036 site-packages\torch\_dynamo\convert_frame.py:1125]                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
W1029 11:03:09.496000 2036 site-packages\torch\_dynamo\convert_frame.py:1125]   File "C:\Users\ZGMF-X42S\miniconda3\envs\test2\Lib\site-package

{'train': 8.375, 'eval': 8.375}


In [22]:
# Optimizer for minimizing the cross-entropy loss 

p_dict = {p_name: p for p_name, p in model.named_parameters() if p.requires_grad}

# Only those with multiple dimensions will benefit from the weight decay
weight_decay_p = [p for n, p in p_dict.items() if p.dim() >= 2]

no_weight_decay_p = [p for n, p in p_dict.items() if p.dim() <2]


optimizer_groups = [
    {'params': weight_decay_p, 'weight_decay': weight_decay},
    {'params': no_weight_decay_p, 'weight_decay': 0.0}
]

optimizer = torch.optim.AdamW(optimizer_groups, lr = lr, betas = (0.9, 0.99))

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_iters, eta_min = lr / 10) 

start_iteration = 0
best_val_loss = float('inf') # Track the best validation loss value











In [23]:
# Loading Checkpoints


# def load_checkpoint(path):
#     print('LLM - Loading Model')
#     checkpoint = torch.load(path)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     iteration = checkpoint['iteration']
#     loss = checkpoint['loss']
#     print(f"Loaded iter {iteration} with loss {loss}")
#     return iteration, loss


# if os.path.exists(f"{checkpoint_dir}/{checkpoint_fn}") and load_pretrained:
#     checkpoint_path = f"{checkpoint_dir}/{checkpoint_load_fn}"
#     start_iteration, loss = load_checkpoint(checkpoint_path)
#     best_val_loss = loss


def load_checkpoint(path):
    print('LLM - Loading Model')
    try:
        checkpoint = torch.load(path)
        print("\nCheckpoint contents:")
        for key in checkpoint.keys():
            print(f"- {key}")
            
        if 'model_state_dict' in checkpoint:
            checkpoint_state = checkpoint['model_state_dict']
            
            # Try to handle OptimizedModule wrapper
            if hasattr(model, '_orig_mod'):
                print("Detected OptimizedModule wrapper, attempting to load into _orig_mod")
                try:
                    # Try loading into the wrapped module
                    model._orig_mod.load_state_dict(checkpoint_state)
                    print("Successfully loaded state into _orig_mod")
                except Exception as e:
                    print(f"Failed to load into _orig_mod: {str(e)}")
                    # If that fails, try to modify the state dict keys
                    modified_state = {"_orig_mod." + k: v for k, v in checkpoint_state.items()}
                    try:
                        model.load_state_dict(modified_state)
                        print("Successfully loaded state with modified keys")
                    except Exception as e2:
                        print(f"Failed to load with modified keys: {str(e2)}")
                        raise
            else:
                # Regular loading if no wrapper
                model.load_state_dict(checkpoint_state)
                
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        iteration = checkpoint['iteration']
        loss = checkpoint['loss']
        print(f"Loaded iter {iteration} with loss {loss}")
        return iteration, loss
        
    except Exception as e:
        print(f"\nDetailed error information:")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {str(e)}")
        
        # Print some debugging information
        if 'model_state_dict' in checkpoint:
            print("\nFirst few checkpoint keys:")
            for k in list(checkpoint['model_state_dict'].keys())[:5]:
                print(f"- {k}")
            
            print("\nFirst few model state_dict keys:")
            for k in list(model.state_dict().keys())[:5]:
                print(f"- {k}")
        raise

if os.path.exists(f"{checkpoint_dir}/{checkpoint_load_fn}") and load_pretrained:
    checkpoint_path = f"{checkpoint_dir}/{checkpoint_load_fn}"
    print(f"Loading checkpoint from: {checkpoint_path}")
    start_iteration, loss = load_checkpoint(checkpoint_path)
    best_val_loss = loss

# DEBUGGING MODULE
# def load_checkpoint(path):
#     print('LLM - Loading Model')
#     try:
#         # Check if path exists
#         if not os.path.exists(path):
#             print(f"Checkpoint path not found: {path}")
#             return 0, float('inf')
            
#         checkpoint = torch.load(path)
#         model.load_state_dict(checkpoint['model_state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#         iteration = checkpoint['iteration']
#         loss = checkpoint['loss']
#         print(f"Loaded iter {iteration} with loss {loss}")
#         return iteration, loss
#     except Exception as e:
#         print(f"Error loading checkpoint: {e}")
#         return 0, float('inf')

# # Fix path construction
# checkpoint_path = os.path.join(checkpoint_dir, checkpoint_fn)
# # or
# # checkpoint_path = f"{checkpoint_dir}/{checkpoint_fn}"

# print(f"Checking path: {checkpoint_path}")
# print(f"Path exists: {os.path.exists(checkpoint_path)}")
# print(f"load_pretrained value: {load_pretrained}")

# if os.path.exists(checkpoint_path) and load_pretrained:
#     print("Loading checkpoint...")
#     start_iteration, loss = load_checkpoint(checkpoint_path)
#     best_val_loss = loss
# else:
#     print("No checkpoint loaded because:")
#     if not os.path.exists(checkpoint_path):
#         print(f"- Checkpoint path does not exist: {checkpoint_path}")
#     if not load_pretrained:
#         print("- load_pretrained is False")



Loading checkpoint from: models//latest.pt
LLM - Loading Model


C:\Users\ZGMF-X42S\AppData\Local\Temp\ipykernel_2036\3390700552.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)



Checkpoint contents:
- model_state_dict
- optimizer_state_dict
- loss
- iteration
Detected OptimizedModule wrapper, attempting to load into _orig_mod
Failed to load into _orig_mod: Error(s) in loading state_dict for GPT:
	Missing key(s) in state_dict: "embeddings.weight", "positions.weight", "blocks.0.ma.heads.0.tril", "blocks.0.ma.heads.0.queries.weight", "blocks.0.ma.heads.0.queries.bias", "blocks.0.ma.heads.0.keys.weight", "blocks.0.ma.heads.0.keys.bias", "blocks.0.ma.heads.0.values.weight", "blocks.0.ma.heads.0.values.bias", "blocks.0.ma.heads.1.tril", "blocks.0.ma.heads.1.queries.weight", "blocks.0.ma.heads.1.queries.bias", "blocks.0.ma.heads.1.keys.weight", "blocks.0.ma.heads.1.keys.bias", "blocks.0.ma.heads.1.values.weight", "blocks.0.ma.heads.1.values.bias", "blocks.0.ma.heads.2.tril", "blocks.0.ma.heads.2.queries.weight", "blocks.0.ma.heads.2.queries.bias", "blocks.0.ma.heads.2.keys.weight", "blocks.0.ma.heads.2.keys.bias", "blocks.0.ma.heads.2.values.weight", "blocks.0.ma.he

RuntimeError: Error(s) in loading state_dict for OptimizedModule:
	Missing key(s) in state_dict: "_orig_mod.embeddings.weight", "_orig_mod.positions.weight", "_orig_mod.blocks.0.ma.heads.0.tril", "_orig_mod.blocks.0.ma.heads.0.queries.weight", "_orig_mod.blocks.0.ma.heads.0.queries.bias", "_orig_mod.blocks.0.ma.heads.0.keys.weight", "_orig_mod.blocks.0.ma.heads.0.keys.bias", "_orig_mod.blocks.0.ma.heads.0.values.weight", "_orig_mod.blocks.0.ma.heads.0.values.bias", "_orig_mod.blocks.0.ma.heads.1.tril", "_orig_mod.blocks.0.ma.heads.1.queries.weight", "_orig_mod.blocks.0.ma.heads.1.queries.bias", "_orig_mod.blocks.0.ma.heads.1.keys.weight", "_orig_mod.blocks.0.ma.heads.1.keys.bias", "_orig_mod.blocks.0.ma.heads.1.values.weight", "_orig_mod.blocks.0.ma.heads.1.values.bias", "_orig_mod.blocks.0.ma.heads.2.tril", "_orig_mod.blocks.0.ma.heads.2.queries.weight", "_orig_mod.blocks.0.ma.heads.2.queries.bias", "_orig_mod.blocks.0.ma.heads.2.keys.weight", "_orig_mod.blocks.0.ma.heads.2.keys.bias"

In [24]:
# Inference 

if inference == True: 
    model.eval()
    while True:
        qs = input('Enter text (q to quit): ')
        if qs == "":
            continue
        if qs == "q":
            break
        generate_sample(qs)


Enter text (q to quit):  I go to school by bus


I go to school by bus books colon~ waysexev Chic joined chart estungient market Danielcience Jes UKiol enginealf Khem suggestyr Hel clos being Wal agre phrey Hist Blackended Bang tooilesston Persight sing Walpleming Khatithingimbround heldylesp near Angelesired medicalanks politician Japangest Os COVID building computer


Enter text (q to quit):  q


In [62]:
# Training Loop

try:
    for i in tqdm(range(start_iteration, train_iters)):
        print(f"Iteration: {i}")
        xb, yb = get_batch('train')
        logits, loss = model(xb, yb)
    
        # Evaluating loss
        if (i % eval_interval == 0 or i == train_iters-1):
            l = calculate_loss()
            print(f"\n{i}: train loss: {l['train']} / val loss: {l['eval']}")
            # generate_sample('Once upon a time')
    
        if l['eval'] < best_val_loss:
            best_val_loss = l['eval']
            print('[CHECKPOINT]: Saving with loss: ', best_val_loss)
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_val_loss,
                'iteration': i,
            }, checkpoint_dir + checkpoint_fn)
    
        if wandb_log:
            wandb.log({
                'loss/train': l['train'],
                'loss/val': l['eval'],
                'lr': scheduler.get_last_lr()[0],
            },
            step=i)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()

        nn.utils.clip_grad_norm(model.parameters(), max_norm=grad_clip)

        optimizer.step()
        scheduler.step()

    if wandb_log:
        wandb.finish()

except KeyboardInterrupt:
    print('Training interrupted. Cleaning up......')

finally:
    # Release GPU memory
    print('GPU memory released')
    # sys.exit(0)




# try:
#     for i in tqdm(range(start_iteration, train_iters)):
#         xb, yb = get_batch('train')
#         logits, loss = model(xb, yb)
    
#         # Evaluating loss
#         if (i % eval_interval == 0 or i == train_iters-1):
#             l = calculate_loss()
#             print(f"\n{i}: train loss: {l['train']} / val loss: {l['eval']}")
#             # generate_sample('Once upon a time')
    
#         if l['eval'] < best_val_loss:
#             best_val_loss = l['eval']
#             print('[CHECKPOINT]: Saving with loss: ', best_val_loss)
#             torch.save({
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'loss': best_val_loss,
#                 'iteration': i,
#             }, checkpoint_dir + checkpoint_fn)
    
    
#         if wandb_log:
#             wandb_log({
#                 'loss/train': l['train'],
#                 'loss/val': l['eval'],
#                 'lr': scheduler.get_last_lr()[0],
#             },
#             step = i)

#         optimizer.zero_grad(set_to_none = True)
#         loss.backward()

#         nn.utils.clip_grad_norm(model.parameters(), max_norm = grad_clip)

#         optimizer.step()
#         scheduler.step()

#     if wandb_log:
#         wandb.finish()

# except KeyboardInterrupt:
#     print('Training interrupted. Cleaning up......')

# finally:
#     # Release GPU memory
#     print('GPU memory released')
#     # sys.exit(0)

# # torch.cuda.empty_cache()






  0%|                                                                                                 | 0/100000 [00:00<?, ?it/s]

Iteration: 0

0: train loss: 8.375 / val loss: 8.375
[CHECKPOINT]: Saving with loss:  8.375


C:\Users\ZGMF-X42S\AppData\Local\Temp\ipykernel_15204\2663719481.py:36: FutureWarning: `torch.nn.utils.clip_grad_norm` is now deprecated in favor of `torch.nn.utils.clip_grad_norm_`.
  nn.utils.clip_grad_norm(model.parameters(), max_norm=grad_clip)
  0%|                                                                                      | 1/100000 [00:02<65:50:08,  2.37s/it]

Iteration: 1


  0%|                                                                                      | 2/100000 [00:02<30:48:40,  1.11s/it]

...

Iteration: 1377


  1%|█▏                                                                                  | 1378/100000 [08:33<8:17:18,  3.31it/s]

Iteration: 1378


  1%|█▏                                                                                 | 1378/100000 [08:33<10:12:50,  2.68it/s]


Training interrupted. Cleaning up......
GPU memory released


In [63]:
!nvidia-smi


Mon Oct 28 23:31:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.80                 Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   81C    P0              79W /  80W |   4201MiB /  6144MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--